In [1]:
import pandas as pd
import numpy as np

def min_mc_weight(row):
    # Use 'average FC' if 'Min_weight' is NaN (blank)
    min_weight = row['Min_weight'] if pd.notna(row['Min_weight']) else row['average FC']
    
    return (min_weight * 1219) / row['width']

In [2]:
uat = "1110"
# FINISH GOODS - add forecast + stockend (needcut > 0) + coil center priority
finish_csv = pd.read_csv(f"../data/Slit RequestSRO-{uat}.csv")
finish_csv['3rd Priority'] = finish_csv['3rd Priority'].astype("object")
finish_csv['1st Priority'] = finish_csv['1st Priority'].astype("object")
finish_csv['2nd Priority'] = finish_csv['2nd Priority'].astype("object")
finish_csv.drop(columns='Customer',inplace=True)
finish_csv.rename(columns={'Customer Short Name':"Customer"},inplace=True)

finish_csv.drop(columns=['Month',"Year"],inplace=True)
finish_csv['Length'] = finish_csv['Length'].str.upper()
finish_csv["MC Code"] = finish_csv.apply(lambda row: f"{row['Maker']} {row['Spec']} {row['Thickness']:.2f}", axis=1)
# finish_csv = finish_csv[(finish_csv['Length'] == 'C') |(finish_csv['CUT COIL'] =="yes")]

# Fix standard
finish_csv['Standard'] = finish_csv['Standard'].str.lower()

finish_csv["Customer_FG"] = (finish_csv['Customer'].astype(object)+"-"+
                                finish_csv['FG Code'].astype(object))
finish_csv['Stockend Qt'] = finish_csv['Stockend Qt'].astype(float)

In [3]:
# STOCK END
stock_end = pd.read_excel("../data/Stock End Details.xlsx")
stock_end.drop(columns=['(Do Not Modify) Row Checksum','(Do Not Modify) Modified On','(Do Not Modify) StockEndDetail',
'(Do Not Modify) Modified On','(Do Not Modify) Row Checksum','BP Code'],inplace=True)
stock_end.rename(columns={"Customer Short Name (BP Code) (Customer Master)":"Customer",
                        'Width (FG Code) (Material Code Master (FG codes))':"Width",
                        "Length (FG Code) (Material Code Master (FG codes))":"Length",
                        "Stock End Qt":"Stockend Qt"
                        },
                        inplace=True)

stock_end = stock_end[(stock_end['Length'] == 'C') |(stock_end['Length'] == '1219')]
stock_end["Customer_FG"] = (stock_end['Customer'].astype(object)+"-"+
                                stock_end['FG Code'].astype(object))
total_stock_end = stock_end.groupby('Customer_FG')['Stockend Qt'].sum().reset_index()
substock_end = stock_end[['FG Code','Customer','Customer_FG']]

total_stock_end['Stockend Qt'] = total_stock_end['Stockend Qt'].astype(float)
total_stock_end['Need Cut'] = total_stock_end['Stockend Qt']

merged_stock_end = pd.merge(total_stock_end, substock_end, on="Customer_FG",how="left")
merged_stock_end_filtered = merged_stock_end[~merged_stock_end['Customer_FG'].isin(finish_csv['Customer_FG'])]

concatenated_po = pd.concat([finish_csv, merged_stock_end_filtered], ignore_index=True)

c:\Users\thuduong\AppData\Local\anaconda3\envs\optima\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
# COIL CENTER PRIORITY
coil_center_prio = pd.read_excel("../data/Coil Center Priorities.xlsx")
coil_center_prio['3rd Priority'] = coil_center_prio['3rd Priority'].astype("object")
coil_center_prio['1st Priority'] = coil_center_prio['1st Priority'].astype("object")
coil_center_prio['2nd Priority'] = coil_center_prio['2nd Priority'].astype("object")

# Drop Standard and take Calculate Standard
coil_center_prio.drop(columns=['Standard'], inplace = True)
coil_center_prio['Standard'] = coil_center_prio['Calculate Standard'].str.lower()
coil_center_prio.drop(columns=['Calculate Standard'], inplace = True)

coil_center_prio["Customer_FG"] = (coil_center_prio['Customer'].astype(object)+"-"+
                                coil_center_prio['FG Code'].astype(object))
# coil_center_prio.drop(columns=['BP code'],inplace=True)
coil_center_prio = coil_center_prio.apply(lambda col: col.fillna('') if col.dtype == 'object' else col.fillna(np.nan))
coil_center_prio

,Customer,FG Code,1st Priority,2nd Priority,3rd Priority,Min,Max,Standard,Customer_FG
0,VPIC1,POSCOVN SPCC-SD 1.00X143.5XC,HSC,,,NaN,NaN,,VPIC1-POSCOVN SPCC-SD 1.00X143.5XC
1,VPIC1,CSVC JSH440W-PO 2.60X180XC,HSC,,,NaN,NaN,,VPIC1-CSVC JSH440W-PO 2.60X180XC
2,VPIC1,POSCO JSH270C-PO 2.60X219XC,NQS,,,NaN,NaN,,VPIC1-POSCO JSH270C-PO 2.60X219XC
3,VPIC1,POSCOVN SPCC-SD 1.00X58XC,HSC,,,NaN,NaN,,VPIC1-POSCOVN SPCC-SD 1.00X58XC
4,TMW,CSVC JSH270C-PO 2.00X115XC,NQS,HSC,,NaN,NaN,,TMW-CSVC JSH270C-PO 2.00X115XC
...,...,...,...,...,...,...,...,...,...
2544,XLDA,POSCOVN SPCC-SD 1.40X1219X2200,HSC,,,NaN,NaN,,XLDA-POSCOVN SPCC-SD 1.40X1219X2200
2545,XLDA,CSVC JSH270C-PO 2.00X67XC,NQS,HSC,,NaN,NaN,,XLDA-CSVC JSH270C-PO 2.00X67XC
2546,XLDA,CSVC JSH440W-PO 1.60X183XC,HSC,,,500.0,NaN,small,XLDA-CSVC JSH440W-PO 1.60X183XC
2547,GSK,JFE SUS410M 3.70X1047XC,NQS,,,NaN,NaN,,GSK-JFE SUS410M 3.70X1047XC


In [7]:
# Merge the DataFrames on the 'Key' column
merged_concatenated_po = pd.merge(concatenated_po, coil_center_prio, on='Customer_FG', how='left',suffixes=('_po', '_cc'))
# Fill NaN values in df1 columns with corresponding values from df2 columns
for col in coil_center_prio.columns:
    if col != 'Customer_FG':
        merged_concatenated_po[col + '_po'] = merged_concatenated_po[col + '_po'].combine_first(merged_concatenated_po[col + '_cc'])
        merged_concatenated_po.drop(columns=[col + '_cc'], inplace=True)

# Rename columns to remove suffixes
merged_concatenated_po.columns = [col.replace('_po', '') for col in merged_concatenated_po.columns]

In [8]:
# FORECAST
forecast = pd.read_excel("../data/Forecast from Client.xlsx")
forecast.drop(columns=['(Do Not Modify) FCClient','(Do Not Modify) Row Checksum','(Do Not Modify) Modified On',
'FCID','(Do Not Modify) Modified On','(Do Not Modify) FCClient','(Do Not Modify) Row Checksum','BP Code'],inplace=True)
forecast.rename(columns={"Customer Short Name (BP Code) (Customer Master)":"Customer"},inplace=True)
forecast['Target Month'] = pd.to_datetime(forecast['Target Month'], format='%Y-%m')
forecast["Customer_FG"] = (forecast['Customer'].astype(object)+"-"+
                                forecast['FG Code'].astype(object))

# Assume that current month = 1, fc1==2, fc2==3, fc3==4, 
forecast_fc1 = forecast[forecast['Forecast date'].dt.month == 2]
latest_indices_fc1 = forecast_fc1.groupby('Customer_FG')['Target Month'].idxmax()
latest_records_fc1 = forecast_fc1.loc[latest_indices_fc1,['Customer_FG','Forecast Qty']]
latest_records_fc1.rename(columns={'Forecast Qty':"fc1"},inplace=True)

forecast_fc2 = forecast[forecast['Forecast date'].dt.month == 3]
latest_indices_fc2 = forecast_fc2.groupby('Customer_FG')['Target Month'].idxmax()
latest_records_fc2 = forecast_fc2.loc[latest_indices_fc2,['Customer_FG','Forecast Qty']]
latest_records_fc2.rename(columns={'Forecast Qty':"fc2"},inplace=True)

forecast_fc3 = forecast[forecast['Forecast date'].dt.month == 4]
latest_indices_fc3 = forecast_fc3.groupby('Customer_FG')['Target Month'].idxmax()
latest_records_fc3 = forecast_fc3.loc[latest_indices_fc3,['Customer_FG','Forecast Qty','Maker','Spec','Thickness','Width','Length']]
latest_records_fc3.rename(columns={'Forecast Qty':"fc3"},inplace=True)

c:\Users\thuduong\AppData\Local\anaconda3\envs\optima\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
merged_forecast = latest_records_fc1.merge(latest_records_fc2, on='Customer_FG', how='outer').merge(latest_records_fc3, on='Customer_FG', how='outer')
merged_forecast['average FC'] = merged_forecast[['fc1', 'fc2', 'fc3']].mean(axis=1)

merged_po_forecast = pd.merge(merged_concatenated_po, merged_forecast, on='Customer_FG', how='left',suffixes=('_po', '_fc'))
# Fill NaN values in df1 columns with corresponding values from df2 columns
for col in merged_forecast.columns:
    if col not in ['Customer_FG','fc1','fc2','fc3','average FC']:
        merged_po_forecast[col + '_po'] = merged_po_forecast[col + '_po'].combine_first(merged_po_forecast[col + '_fc'])
        merged_po_forecast.drop(columns=[col + '_fc'], inplace=True)
# Rename columns to remove suffixes
merged_po_forecast.columns = [col.replace('_po', '') for col in merged_po_forecast.columns]

# DROP IF STANDARD OR 1ST EMPTY
merged_po_forecast = merged_po_forecast.dropna(subset=['Standard', '1st Priority'])

# SAVE
merged_po_forecast.drop_duplicates(inplace=True)
merged_po_forecast.rename(columns={"Customer":'customer_name',
                                    "FG Code":"fg_codes",
                                    "Maker":'maker',
                                    "Spec":'spec',
                                    "Thickness":'thickness',
                                    "Length":'length',
                                    "Width":'width',
                                    "Need Cut":'need_cut',
                                    "Standard":'standard',
                                    "Min":'Min_weight',
                                    "Max": "Max_weight",
                                    }, inplace = True)
merged_po_forecast['average FC'] = merged_po_forecast['average FC'].fillna(0)

merged_po_forecast.reset_index(inplace=True)
merged_po_forecast.rename(columns={'index': 'order_id'}, inplace=True)
merged_po_forecast["Min_MC_weight"] = merged_po_forecast.apply(min_mc_weight, axis=1)

In [11]:
# FIX STANDARD AFTER MERGING ALL TABLES
# Define conditions
conditions = [
    (merged_po_forecast['Min_MC_weight'] <= 3500) & (merged_po_forecast['standard'] == 'na'),
    (merged_po_forecast['Min_MC_weight'] > 3500) & (merged_po_forecast['Min_MC_weight'] <= 7000) & (merged_po_forecast['standard'] == 'na')
]

# Define corresponding outputs
choices = ['small', 'medium']

# Apply the conditions
merged_po_forecast['standard'] = np.select(conditions, choices, default='big')

#Fix for STEEL SHEET
merged_po_forecast.loc[(merged_po_forecast['length'] != '1219') & (merged_po_forecast['standard'] == 'na'), 'standard'] = 'small'

In [12]:
merged_po_forecast.to_excel(f'../data/finish_uat_{uat}_v1.xlsx',index=False)

In [41]:
# ------------

In [ ]:
# MOTHER COIL - filter status + overlap MC code
import pandas as pd

mother_coil = pd.read_excel("../data/MC Inventories.xlsx")
mother_coil.drop(columns=['(Do Not Modify) MC_Inventory','(Do Not Modify) Row Checksum','(Do Not Modify) Modified On',
'(Do Not Modify) Modified On','Inspection No','Grade','HTV Comment','Prod. Date','Customer Short Name (BP Code) (Customer Master)','Ngày tạo'],inplace=True)
mother_coil.rename(columns={
                            "Receiving Date":'receiving_date',
                            "Inventory ID":'inventory_id',
                            'Coil Center':'warehouse',
                            "Maker":'maker',
                            "Spec":'spec',
                            "Thickness":'thickness',
                            "Length":'length',
                            "Width":'width',
                            "Status":'status',
                            "Remark":'remark'
                            },inplace=True)

c:\Users\thuduong\AppData\Local\anaconda3\envs\optima\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [43]:
from datetime import datetime

mother_coil['weight'] = mother_coil["Weight"] - mother_coil["Usage Weight"].fillna(0)
mother_coil["MC Code"] = mother_coil.apply(lambda row: f"{row['maker']} {row['spec']} {row['thickness']:.2f}", axis=1)
mother_coil['Receiving Date'] = pd.to_datetime(mother_coil['Receiving Date'])
# Assuming mother_coil is your DataFrame
mother_coil['receiving_date'] = mother_coil['Receiving Date'].apply(lambda x: x.isoformat())

In [44]:
filtered_mother_coil = mother_coil[mother_coil['status'].isin(['M:RAW MATERIAL', 'Z:SEMI MCOIL', 'R:REWIND'])]
filtered_mother_coil=filtered_mother_coil.reset_index(drop=True)
filtered_mother_coil = filtered_mother_coil.drop_duplicates(subset='inventory_id', keep='first')
filtered_mother_coil.to_excel("../data/mother_coil_uat.xlsx", index= False)

In [45]:
filtered_mother_coil

,inventory_id,warehouse,FG Code,maker,spec,thickness,length,width,Weight,Qty,Usage Status,Usage Weight,status,remark,Receiving Date,weight,MC Code,receiving_date
0,A1,HSC,POSCO SPHT-1-PO 1.53,POSCO,SPHT-1-PO,1.53,NaN,1180.0,11970,1,NaN,NaN,M:RAW MATERIAL,NaN,2022-12-12 07:00:00,11970.0,POSCO SPHT-1-PO 1.53,2022-12-12T07:00:00
2,A2,HSC,POSCO SPHT-1-PO 1.53,POSCO,SPHT-1-PO,1.53,NaN,785.0,7805,1,NaN,NaN,M:RAW MATERIAL,NaN,2024-10-14 07:00:00,7805.0,POSCO SPHT-1-PO 1.53,2024-10-14T07:00:00
3,A3,NQS,POSCOVST SUS409L-2D 1.50,POSCOVST,SUS409L-2D,1.50,NaN,1049.0,7087,1,NaN,NaN,M:RAW MATERIAL,NaN,2024-10-14 07:00:00,7087.0,POSCOVST SUS409L-2D 1.50,2024-10-14T07:00:00
4,A4,NQS,POSCOVST SUS409L-2D 1.50,POSCOVST,SUS409L-2D,1.50,NaN,1049.0,8000,1,NaN,NaN,M:RAW MATERIAL,NaN,2024-11-01 07:00:00,8000.0,POSCOVST SUS409L-2D 1.50,2024-11-01T07:00:00
5,A5,NQS,POSCOVST SUS409L-2D 1.50,POSCOVST,SUS409L-2D,1.50,NaN,1049.0,7500,1,NaN,NaN,M:RAW MATERIAL,NaN,2024-11-01 07:00:00,7500.0,POSCOVST SUS409L-2D 1.50,2024-11-01T07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,TP24CH003758,HSC,CSC JSC590R-SD 1.20X1219XC,CSC,JSC590R-SD,1.20,C,1219.0,4925,1,NaN,NaN,R:REWIND,HTV-S2-241020-2,2024-10-30 07:00:00,4925.0,CSC JSC590R-SD 1.20,2024-10-30T07:00:00
1284,TP24CH003932,HSC,CSVC JSH440W-PO 1.60X893XC,CSVC,JSH440W-PO,1.60,C,893.0,7465,1,NaN,NaN,Z:SEMI MCOIL,1C24TNS-553,2024-11-05 07:00:00,7465.0,CSVC JSH440W-PO 1.60,2024-11-05T07:00:00
1285,TP24CH003943,HSC,CSC JSC590R-SD 1.60X1219XC,CSC,JSC590R-SD,1.60,C,1219.0,9554,1,NaN,NaN,R:REWIND,HTV-S2-240930-2,2024-11-06 07:00:00,9554.0,CSC JSC590R-SD 1.60,2024-11-06T07:00:00
1286,TP24CH003958,HSC,CSC JSH590R-PO 2.00X1219XC,CSC,JSH590R-PO,2.00,C,1219.0,8713,1,NaN,NaN,R:REWIND,HTV-S2-241130-1,2024-12-17 07:00:00,8713.0,CSC JSH590R-PO 2.00,2024-12-17T07:00:00
